In [ ]:
import pyfits,glob,time
from subprocess import call
#from ds9 import *
from numpy import *
import numpy as np
from scipy.ndimage import zoom,shift
import scipy.ndimage as snd
from matplotlib.pyplot import *
import seaborn as sb
from itertools import *
from matplotlib.colors import LogNorm
%matplotlib inline

In [ ]:
def mosaic_combine_ITL(filename):
    "Takes in a multi-extension mosaic fits file and the dimensions of the mosaic and returns an array with all chips combined"
    dim_x,dim_y=8,2
    hdr0 = pyfits.getheader(filename,0)
    numccds = hdr0['nextend']
    sizex,sizey = hdr0['IMG_COLS'],hdr0['IMG_ROWS']
    overscanx,overscany = hdr0['OVR_COLS'],hdr0['OVR_ROWS']
    prescanx,prescany = hdr0['PRE_COLS'],hdr0['PRE_ROWS']
    sizex,sizey = sizex,sizey
    
    ccdarray = np.zeros((sizey*dim_y,sizex*dim_x),dtype='float')
        
    for ccdnum in range(numccds):
        ccdseg = pyfits.getdata(filename,ccdnum+1)
        med_overscan=median(ccdseg[:,-overscanx:])
        ccdseg=ccdseg[prescany:-overscany,prescanx:-overscanx]-med_overscan
        hdr = pyfits.getheader(filename,ccdnum+1)
        detsec = hdr['DETSEC'].strip("[']").split(",")
        x1 = int(detsec[0].strip("'").split(":")[0])
        x2 = int(detsec[0].strip("'").split(":")[1])
        y1 = int(detsec[1].strip("'").split(":")[0]) 
        y2 = int(detsec[1].strip("'").split(":")[1])
        # x1, x2, y1, y2 are the extents of each segment
        ltm11 = hdr['LTM1_1']
        ltm22 = hdr['LTM2_2']
        # LTM1_1 and LTM2_2 are +/-1 and tell whether the direction is inverted 
        if ltm22 < 0:
            y1,y2 = y2,y1
            ccdseg=ccdseg[::-1,:]
        y1 = y1 - 1
        x1 = sizex * dim_x - x1
        x2 = sizex * dim_x - x2
        if ltm11 > 0:
            x1,x2 = x2,x1
        x2 = x2 + 1
        ccdarray[y1:y2,x1:x2]=ccdseg
    # The code here keeps the convention of having 0,0 in the lower left.
    # This is different from ds9, which has 0,0 in the lower right.
    return ccdarray

def make_ell_reg_from_xy(filename):
    """Takes in X & Y coordinates and a text tag and outputs a DS9 region file"""
    foocat=pyfits.getdata(filename+'.cat',"LDAC_OBJECTS")
    xreg,yreg=foocat['XWIN_IMAGE'],foocat['YWIN_IMAGE']
    radius1=7*np.ones(len(xreg))
    radius2=foocat['B_IMAGE']/foocat['A_IMAGE']*radius1
    anglereg=foocat['THETA_IMAGE']
    text_tag=['']*len(xreg)
    reg_filename=filename+'.reg'
    f = open(reg_filename,'w')
    f.write('image'+' \n')
    for i in range(len(xreg)):
        thetext=text_tag[i]
        f.write('ellipse '+str(xreg[i])+' '+str(yreg[i])+' '+str(radius1[i])+' '+str(radius2[i])+
                ' '+str(anglereg[i])+' #text="'+str(thetext)+'"\n')
    f.close()

    
def run_sextractor_on_mosaic(fitsfilename):
    #from mydefs import make_ell_reg_from_xy
    #from mydefs import make_reg_from_ldac
    import pyfits,numpy
    from subprocess import call
    print topdir
    configfile=topdir+'sextractor/30-micron-pinhole.sex'
    paramfile=topdir+'sextractor/30-micron-default-array_dither.param'
    hdr0=pyfits.open(fitsfilename)[0].header
    pdiode=hdr0['MONDIODE']
    exptime=hdr0['EXPTIME']
    fluxval=float(pdiode)*float(exptime)
    minadu=5e10*fluxval
    outname=fitsfilename
    callstring=["sex",outname,"-c",configfile,"-CATALOG_NAME",outname+'.cat',"-CATALOG_TYPE","FITS_LDAC",
               "-CHECKIMAGE_NAME",outname[:-4]+"back.fits"+','+outname[:-4]+"rms.fits",
               "-CHECKIMAGE_TYPE","BACKGROUND,BACKGROUND_RMS",
               "-DETECT_THRESH","10","-DETECT_MINAREA","10","-THRESH_TYPE","RELATIVE","-DETECT_MAXAREA","400",
               "-ANALYSIS_THRESH","2"]
    test=call(callstring)
    make_ell_reg_from_xy(fitsfilename)
    #make_ell_reg_from_xy(outname)
    #call(["rm",outname])
    #print callstring
    

def make_stamp_stack_subpix(cat,imagename,obj_inds,halfstampsize,nzoom, means = None, stds = None ):
    # If list are supplied for the mean and std of the spot max, append to them
    if means is not None and stds is not None:
        CalculateMaxs = True
    else:
        CalculateMaxs = False
    if CalculateMaxs:
        maxs = []
    sz=halfstampsize
    nobjs=len(obj_inds)
    img=pyfits.getdata(imagename)
    xc,yc=cat['XWIN_IMAGE'],cat['YWIN_IMAGE']
    nstamps=0
    stampstack=np.zeros((sz*nzoom*2,sz*nzoom*2))
    xsize,ysize=3900,3900
    for k in range(nobjs):
        ind=obj_inds[k]
        x,y=xc[ind]+.5,yc[ind]+.5
        if ((x<sz) | (x >xsize-sz-1) | (y<sz) | (y>ysize-sz-1)): 
            continue
        xf,yf=np.floor(x),np.floor(y)
        xshift,yshift=round((x-xf)*(nzoom)),round((y-yf)*(nzoom))
        onestamp=img[int(y-sz):int(y+sz),int(x-sz):int(x+sz)]
        if CalculateMaxs:
            maxs.append(onestamp.max())
        onestamp=onestamp/np.max(onestamp)
        regridstamp=np.zeros((sz*nzoom*2,sz*nzoom*2))
        for i in range(sz*2):
            for j in range(sz*2):
                i0,i1=nzoom*i,nzoom*(i+1)
                j0,j1=nzoom*j,nzoom*(j+1)
                regridstamp[i0:i1,j0:j1]=onestamp[i,j]
        shiftstamp=shift(regridstamp,(-yshift+nzoom,-xshift+nzoom),order=0)
        stampstack+=shiftstamp
        nstamps+=1
    if CalculateMaxs:
        maxs = np.array(maxs)
        means.append(maxs.mean())
        stds.append(maxs.std())
    stampstack=stampstack/nstamps
    return stampstack

def make_mosaic_stamp_subpix(stackfitsname,halfwin,ndiv,nzoom):
    """Takes in a fits file and searches for its catalog, then stacks object images by shifting 
    and medianing. Halfwin is the half size of the window around each object, and ndiv is how many 
    divisions to split the image into"""
    sexcat=pyfits.getdata(stackfitsname+'.cat',"LDAC_OBJECTS")
    fw=(halfwin*2*nzoom)
    xmossize,ymossize=4071,3999#4072,4000
    xgridsz,ygridsz=xmossize/ndiv*1.,ymossize/ndiv*1.
    xgridmins,ygridmins=linspace(0,xmossize,ndiv),linspace(0,ymossize,ndiv)
    allstampstacks=np.zeros(((halfwin*2*nzoom+1)*ndiv,(halfwin*2*nzoom+1)*ndiv))
    for i in range(ndiv):
        xgridmin,xgridmax=i*xgridsz,(i+1)*xgridsz
        for j in range(ndiv):
            ygridmin,ygridmax=j*ygridsz,(j+1)*ygridsz
            g=where((sexcat['XWIN_IMAGE']>xgridmin) & (sexcat['XWIN_IMAGE'] < xgridmax) &
                    (sexcat['YWIN_IMAGE']>ygridmin) & (sexcat['YWIN_IMAGE'] < ygridmax) &
                    (sexcat['FLAGS']==0))[0]
            onestamp=make_stamp_stack_subpix(sexcat,stackfitsname,g,halfwin,nzoom)
            allstampstacks[j*fw:(j+1)*fw,i*fw:(i+1)*fw]=onestamp
    return allstampstacks

def radial_profile(data, center):
    y, x = np.indices((data.shape))
    r = np.sqrt((x - center[0])**2 + (y - center[1])**2)
    r = r.astype(np.int)
    tbin = np.bincount(r.ravel(), data.ravel())
    nr = np.bincount(r.ravel())
    radialprofile = tbin / nr
    rx=linspace(0,halfstampsize*np.sqrt(2),round(halfstampsize*nzoom*np.sqrt(2)))
    return rx,radialprofile 

def moments(data, center):
    y, x = np.indices((data.shape))
    x = (x - center[0] + 0.5) / nzoom
    y = (y - center[1] + 0.5) / nzoom
    mx = (data * x).sum() / data.sum()
    my = (data * y).sum() / data.sum()
    mx2 = (data * (x-mx) * (x-mx)).sum() / data.sum()
    my2 = (data * (y-my) * (y-my)).sum() / data.sum()
    mxy = (data * (x-mx) * (y-my)).sum() / data.sum()
    return [mx, my, mx2, my2, mxy]



In [ ]:
topdir = '/sandbox/lsst/lsst/GUI/'
thedir=topdir+'20170313-bf-30um-VBB60-R/'
%cd $thedir
expnum_root,date='5','20170313'
filelist=sort(glob.glob(thedir+'ITL-3800C-029_spot_spot_'+expnum_root+'??_'+date+'??????_ct.fits'))
print len(filelist)
mos_suffix="whole.fits"

for filename in filelist[:]:
    pyfits.writeto(filename[:-5]+mos_suffix,mosaic_combine_ITL(filename),header=pyfits.getheader(filename,0),clobber='True')
    

In [ ]:
mos_suffix="whole.fits"
date='20170313'
globstring=thedir+'ITL-3800C-029_spot_spot_'+expnum_root+'??_'+date+'??????_ct'+mos_suffix
mosfilelist=sort(glob.glob(globstring))
print len(mosfilelist)

In [ ]:
tstart = time.time()

for i in range(len(mosfilelist)): run_sextractor_on_mosaic(mosfilelist[i])

print time.time() - tstart

In [ ]:
thenum=20
nzoom=10
halfstampsize=5
xmin,xmax=1000,3000
ymin,ymax=1000,3000

cat=pyfits.getdata(mosfilelist[thenum]+'.cat',"LDAC_OBJECTS")
obj_inds=where((cat['XWIN_IMAGE']<xmax) & (cat['YWIN_IMAGE']<ymax) &
           (cat['XWIN_IMAGE']>xmin) & (cat['YWIN_IMAGE']>ymin) & (cat['FLAGS']==0))[0]
stampstack=make_stamp_stack_subpix(cat,mosfilelist[thenum],obj_inds,halfstampsize,nzoom)

figure(figsize=(10,5))
subplot(121)
imshow(stampstack,cmap='rainbow',aspect='equal',origin='lower'),colorbar()
for i in range(1,2*halfstampsize):
    axvline(i*nzoom,color='w')
    axhline(i*nzoom,color='w')
axis('off')


subplot(122)
rx,rp=radial_profile(stampstack,(halfstampsize*nzoom,halfstampsize*nzoom))

plot(rx,rp)
#yscale('log')

foox=np.linspace(0,8,100)
foosig=1.7
fooy=np.exp(-foox**2/foosig**2)
plot(foox,fooy)


In [ ]:
from scipy.special import airy,jv

In [ ]:
figure(figsize=(12,5))
rx,rp=radial_profile(stampstack,(halfstampsize*nzoom,halfstampsize*nzoom))
plot(rx,rp,label='Data stacked')
#yscale('log')

foox=np.linspace(.001,8,100)
foosig=1.8
fooy=np.exp(-foox**2/foosig**2)
plot(foox,fooy,label='Gaussian')
fooy=(jv(1,foox/.9)/foox)**2
plot(foox,fooy/fooy[0],label='Airy')
legend(loc='upper right')
axis([0,5,1e-4,1])

In [ ]:
thenum=20
stackfitsname=mosfilelist[thenum]
ndiv=8
halfwin=5
tstart=time.time()
allstampstacks=make_mosaic_stamp_subpix(stackfitsname,halfwin,ndiv,nzoom)
print time.time()-tstart

In [ ]:
figure(figsize=(15,15))
imshow(allstampstacks,origin='lower',cmap='rainbow',interpolation='None')#,norm=LogNorm(.01,1.1))
grid(False)
axis('off')
for i in range(8): axvline((i+1)*(halfwin*2*nzoom),color='w')
axhline(ndiv*halfwin*nzoom,color='w')
#ti(linspace(0,4000,40))

In [ ]:
nzoom=10
halfstampsize=5
xmin,xmax=1000,3000
ymin,ymax=1000,3000

means = []
stds = []
mx2s = []
my2s = []
mxys = []
for thenum in range(len(mosfilelist)): 
    cat=pyfits.getdata(mosfilelist[thenum]+'.cat',"LDAC_OBJECTS")
    obj_inds=where((cat['XWIN_IMAGE']<xmax) & (cat['YWIN_IMAGE']<ymax) &
           (cat['XWIN_IMAGE']>xmin) & (cat['YWIN_IMAGE']>ymin) & (cat['FLAGS']==0))[0]
    stampstack=make_stamp_stack_subpix(cat,mosfilelist[thenum],obj_inds,halfstampsize,nzoom,means=means,stds=stds)
    [mx, my, mx2, my2, mxy] = moments(stampstack,(halfstampsize*nzoom,halfstampsize*nzoom))
    mx2s.append(mx2)
    my2s.append(my2)
    mxys.append(mxy)
    print "Finished filenumber %d"%thenum



In [ ]:
from scipy import stats
gain = 5.0
plotmeans = np.array(means) * gain
figure()
title("Brighter-Fatter of Drizzled 30 micron Spots")
scatter(plotmeans,mx2s,color = 'green', label = 'X-Moment')
scatter(plotmeans,my2s,color = 'red', label = 'Y-Moment')
slope, intercept, r_value, p_value, std_err = stats.linregress(plotmeans[10:40],mx2s[10:40])
xplot=linspace(-5000.0,200000.0,100)
yplot = slope * xplot + intercept
plot(xplot, yplot, color='blue', lw = 2, ls = '--')
tslope = slope * 100.0 * 50000.0
text(2000.0,1.35,"X Slope = %.2f %% per 50K e-"%tslope, fontsize = 12)
slope, intercept, r_value, p_value, std_err = stats.linregress(plotmeans[10:40],my2s[10:40])
xplot=linspace(-5000.0,200000.0,100)
yplot = slope * xplot + intercept
plot(xplot, yplot, color='black', lw = 2, ls = '--')
tslope = slope * 100.0 * 50000.0
text(2000.0,1.33,"Y Slope = %.2f %% per 50K e-"%tslope, fontsize = 12)
ylim(1.05,1.4)
xlim(0,250000)
xlabel('Central Peak(electrons)')
ylabel('Second Moment')
legend(loc= 'lower right')
savefig(thedir+"Drizzled_BF_11Aug17.png")

In [ ]:
foocats=np.array([pyfits.getdata(fname+'.cat',"LDAC_OBJECTS") for fname in mosfilelist[:]])

In [ ]:
xcsub,ycsub=[],[]
for foocat in foocats:
    xcsub.extend(foocat['XWIN_IMAGE']-np.floor(foocat['XWIN_IMAGE']))
    ycsub.extend(foocat['YWIN_IMAGE']-np.floor(foocat['YWIN_IMAGE']))
xcsub,ycsub=np.array(xcsub),np.array(ycsub)

In [ ]:
nbins,npins=100,len(xcsub)
pct_var=.09
figure(figsize=(12,6))
hist(xcsub,histtype='step',bins=nbins,lw=4)
hist(ycsub,histtype='step',bins=nbins,lw=4)
xlabel('x/y sub')
ylim(npins/nbins*(1-pct_var),npins/nbins*(1+pct_var))
axhline(npins/nbins)
grid(True)